<div class="alert alert-info">
        
# Исследование надёжности заёмщиков 
    
Вопросы, на которые необходимо ответить в ходе исследования

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Как разные цели кредита влияют на его возврат в срок?

</div>

Исследование надёжности заёмщиков  

Описание проекта  

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-info">
    
## Общая информация о данных
    
</div>

In [1]:
import pandas as pd

In [2]:

try:
    df = pd.read_csv('data.csv')
except:
    df = pd.read_csv('/datasets/data.csv')

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<div class="alert alert-info">

## Поиск аномалий, дублиукатов и пустых значений в данных

</div>

<div class="alert alert-info">

### Поиск пропусков

</div>

In [5]:
all_variables = df.shape[0]

days_employed_nan = df['days_employed'].isna().sum()
days_employed_proportion = days_employed_nan / all_variables

total_income_nan = df['total_income'].isna().sum()
total_income_proportion = total_income_nan / all_variables

print('Доля пропущенных значений в столбце days_employed', days_employed_proportion)
print('Доля пропущенных значений в столбце total_income', total_income_proportion)

Доля пропущенных значений в столбце days_employed 0.10099883855981417
Доля пропущенных значений в столбце total_income 0.10099883855981417


<div class="alert alert-info">

### Заполнение пропусков

</div>

In [6]:
#найдем среднее для каждого столбца с пропусками и сохраним в отдельные переменные
days_employed_mean = df['days_employed'].mean()

In [7]:
#затем заменим пропуски в этих переменных на средние значения
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)

In [8]:
#total_income_mean = df['total_income'].mean()
#df['total_income'] = df['total_income'].fillna(total_income_mean)
df['median'] = df.groupby('income_type')['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(df['median'])
df = df.drop(columns = ['median'],axis = 1)

In [9]:
#проверяем, что пропусков в столбцах days_employed и total_income не осталось
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Обработаем отрицательные значения в столбце days_employed
мои предположения, откуда взялись эти данные:

-дни стажа считали с какой-то конкретной даты, и поэтому стаж, начавшийся раньше этой даты почему-то считается отрицательно

-дни стажа отнимали от возраста клиента, а не от его даты начала работы(когда стаж 0 дней)

-невнимательность сотрудника, вносившего данные(это же относится к значению -1 в графе "дети")

на самом деле меня не столько смущают отрицательные значения, сколько значение в 4 строке датасета: 340266.072047, если разделить это число на 365, то выходит стаж больше 932 лет?

<div class="alert alert-info">

### Поиск аномалий

</div>

In [10]:
def unique_values(list_column):
    for column in list_column:
        print('Кол-во уникальных значений в столбце', column, '=', df[column].unique())
        print()
    

In [11]:
list_column = ['children','days_employed','dob_years','education','education_id','family_status','family_status_id',
              'gender','income_type','debt','total_income','purpose']
unique_values(list_column)


Кол-во уникальных значений в столбце children = [ 1  0  3  2 -1  4 20  5]

Кол-во уникальных значений в столбце days_employed = [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]

Кол-во уникальных значений в столбце dob_years = [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

Кол-во уникальных значений в столбце education = ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Кол-во уникальных значений в столбце education_id = [0 1 2 3 4]

Кол-во уникальных значений в столбце family_status = ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Кол-во уникальных значений в столбце family_status_id = [0 1 2 3 4]

К

<div class="alert alert-info">

### Исправление аномалий

</div>

In [12]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].abs()
df.loc[df['children'] == 20, 'children'] = 2

In [13]:
df['total_income'] = df['total_income'].astype(int)

<div class="alert alert-info">

### Поиск и удаление дубликатов

</div>

In [14]:
df.duplicated().sum()

54

In [15]:
#для начала удалим явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

<div class="alert alert-info">

### Поиск и удаление неявных дубликатов

</div>

In [16]:
def counts_values(list_categorys):
    for column in list_categorys:
        print('Категория' , column)
        print( df[column].value_counts())
        print()

In [17]:
list_categorys = ['education','family_status','gender','income_type']
counts_values(list_categorys)

Категория education
среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Категория family_status
женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Категория gender
F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Категория income_type
сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете          

In [18]:
#далее избавимся от неявных дубликатов в столбце education
df['education'] = df['education'].str.lower()

методом duplicated().sum() вычислила кол-во явных дубликатов, для таких целей этот метод подходит.

методом value_counts() проверила все категориальные столбцы, чтобы было видно, есть ли неявные дубликаты, таким образом было видно, что в столбце education значения повторяются с разными регистрами

этим же методом value_counts() проверила остальные столбцы, в них нет неявных дубликатов, однако есть странный тип гендера: XNA. Выяснить по таблице гендер и заполнить вручную не представляется возможным, поэтому оставлю как есть

Дубли могли появиться как в результате технической ошибки(программа-обработчик записала данные одного и того же клиента 2 раза), так и по невнимательности сотрудника

<div class="alert alert-info">

## Создание таблиц со справочной информацией

</div>

In [19]:
education_dict = df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   education     5 non-null      object
 1   education_id  5 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [20]:
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   family_status_id  5 non-null      int64 
 1   family_status     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [21]:
df = df.drop(['education', 'family_status'], axis='columns')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     21471 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education_id      21471 non-null  int64  
 4   family_status_id  21471 non-null  int64  
 5   gender            21471 non-null  object 
 6   income_type       21471 non-null  object 
 7   debt              21471 non-null  int64  
 8   total_income      21471 non-null  int64  
 9   purpose           21471 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB


<div class="alert alert-info">

## Категоризация клиентов по доходу

</div>

In [22]:
#создадим функцию, которая принимает на вход строку датасета и обрабатывает значения в столбце total_income
#в результате функция возвращает категорию в зависиомсти от дохода
def total_income_categorys(row):
    if 0 < row['total_income'] < 30000:
        return 'E'
    elif 30001 < row['total_income'] < 50000:
        return 'D'
    elif 50001 < row['total_income'] < 200000:
        return 'C'
    elif 200001 < row['total_income'] < 1000000:
        return 'B'
    elif row['total_income'] > 1000001:
        return 'A'

In [23]:
#создадим переменную total_income_category и запишем в нее результат работы функции total_income_categorys
df['total_income_category'] = df.apply(total_income_categorys, axis=1)

создадим функцию, которая распределит по категориям нужды клиентов

‘операции с автомобилем’,
‘операции с недвижимостью’,
‘проведение свадьбы’,
‘получение образования’

для начала узнаем какие вообще есть значение в столбце purpose

In [24]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

значение, связанные с проведением свадьбы, можно отфильтровать по подстроке "свадьб"

значения, связанные с образованием, можно отфильтровать по подстроке "образован"

значения, связанные с авто, можно отфильтровать по подстроке "авто"

значени, связанные с покупкой недвижки, можно отфильтровать по подстрокам "недвиж" и "жиль" 

создадим функцию, которая обработает такие подстроки и создать нужные категории

In [25]:
#создадим функцию, которая принимает на вход строку датасета и обрабатывает значения в столбце purpose
#в результате функция возвращает категорию в зависиомсти от запроса клиента на кредит
def purpose_categorys(row):
    if 'свадьб' in row['purpose']:
        return 'проведение свадьбы'
    elif 'образован' in row['purpose']:
        return 'получение образования'
    elif 'авто' in row['purpose']:
        return 'операции с автомобилем'
    elif 'недвиж' in ['purpose'] or 'жиль' in row['purpose']:
        return 'операции с недвижимостью'

In [26]:
#создадим переменную purpose_category и запишем в нее результат работы функции purpose_categorys
df['purpose_category'] = df.apply(purpose_categorys, axis=1)

<div class="alert alert-info">

## Поиск закономерностей в данных

</div>

<div class="alert alert-info">

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

</div>

In [27]:
debt_children = df[['children', 'debt']]
debt_children_corr = debt_children.groupby('children').sum() / debt_children.groupby('children').count() * 100
pd.pivot_table(debt_children_corr, index='children')




,debt
children,
0,7.535266
1,9.163921
2,9.492481
3,8.181818
4,9.756098
5,0.000000


Есть ли зависимость между количеством детей и возвратом кредита в срок?

Зависимость есть, процент просрочки по кредиту находится в диапазоне 7-9%, заемщики без детей всего на 2% реже возвращают кредит с просрочкой

<div class="alert alert-info">

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

</div>

In [28]:
debt_family = df[['family_status_id', 'debt']]
debt_family_corr = debt_family.groupby('family_status_id').sum() / debt_family.groupby('family_status_id').count() * 100
#display(debt_family_corr)
pd.pivot_table(debt_family_corr, index='family_status_id')


,debt
family_status_id,
0,7.542126
1,9.320202
2,6.569343
3,7.112971
4,9.750890


Есть ли зависимость между семейным положением и возвратом кредита в срок

Зависимость есть, процент просрочки по кредиту находится в диапазоне от 6 до почти 10%, самые большие проценты должников у неженатых заемщиков, либо тех, кто живет в гражданском браке

<div class="alert alert-info">

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

</div>

In [29]:
debt_income = df[['total_income_category', 'debt']]
debt_income_corr = debt_income.groupby('total_income_category').sum() / debt_income.groupby('total_income_category').count() * 100
#display(debt_income_corr)
pd.pivot_table(debt_income_corr, index='total_income_category')


,debt
total_income_category,
A,8.000000
B,7.062091
C,8.483034
D,6.000000
E,9.090909


Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Зависимость есть, процент просрочки по кредиту находится в диапазоне от 6 до почти 9%, самыми добропорядочными являются заемщики с достатком от 30001 до 50000. Так же интересен факт, что процент просрочек у заемщиков с наименьшим достатком всего на 1% выше, чем у заевщиков с наибольшим достатком

<div class="alert alert-info">

### Как разные цели кредита влияют на его возврат в срок?

</div>

In [30]:
debt_purpose_corr = df[['purpose_category', 'debt']].groupby('purpose_category').sum() / df[['purpose_category', 'debt']].groupby('purpose_category').count() * 100
#display(debt_purpose_corr)
pd.pivot_table(debt_purpose_corr, index='purpose_category')


,debt
purpose_category,
операции с автомобилем,9.354689
операции с недвижимостью,6.904282
получение образования,9.217738
проведение свадьбы,7.965739


Как разные цели кредита влияют на его возврат в срок?

Зависимость есть, процент просрочки по кредиту находится в диапазоне 7-9%. Чаще всего кредит в срок возвращают заемщики, у которых цель получения кредита связана с недвижимостью.

<div class="alert alert-info">

## Общий вывод:

Была проанализорована таблица заявок на получение кредита для различных групп клиентов на нужды 4 типов:

операции с недвижимостью
операции с автомобилем
получение образования
проведение свадьбы

из датасета удалены дубликаты, устранены артефакты, а так же созданы 2 справочника: семеного положения и уровня образования

Были проведены исследования влияния различных факторов на возврат клиентом кредита в срок. Судя по результатам, кол-во детей, уровень достатка или же причина взятия кредита влияет на показатель, возвращает ли клиент кредит в срок, но во всех случаях процент неуплаты кредита в срок лежит в диапазоне 6-10%
    
</div>

In [31]:
import datetime as dt
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'22.08.2022 21:13'